In [1]:
#Reference to local files
from constants import *



In [4]:
#Import Dependencies 
import subprocess
import json
import os
from dotenv import load_dotenv

from bit import wif_to_key
from web3 import Web3
from bit import *
from web3.middleware import geth_poa_middleware
from eth_account import Account
from pathlib import Path
from getpass import getpass
from bit.network import NetworkAPI

In [6]:
#Check and load .env file information
load_dotenv()
men_key = os.getenv('men_key')
btc_privatekey = os.getenv('btc_privatekey')
eth_privatekey = os.getenv('eth_privatekey')

print(men_key)
print('-----------------------------------')
print(btc_privatekey)
print('-----------------------------------')
print(eth_privatekey)
print('-----------------------------------')

shuffle kitchen snake payment raccoon ignore demand satoshi final aerobic morning mass
-----------------------------------
cMb3pLmrMW9sbnysRiM96dnHw9x7HwSEzgadZd5fTKq42jE5CRCV
-----------------------------------
e83b2a55b98d73ab2d4e8b1ae881e4380a3df4ece7b5708cd0eced8850afd99a
-----------------------------------


In [7]:
#Create a function called `derive_wallets`
def derive_wallets(menkey, coin, numderive):
    command =  'php derive -g --mnemonic="menkey" --cols=path,address,privkey,pubkey --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)


In [9]:
# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {"ETH", "BTCTEST"}

coin_keys = {}
for coin in coins:
    coin_keys[coin] = derive_wallets(men_key, coin, 2)

In [10]:
# Confirm final object
coin_keys

{'BTCTEST': [{'path': "m/44'/0'/0'/0/0",
   'address': '17AqTbYNCrHMo71GuhsukbavvbbX2bcp9a',
   'privkey': 'L2BwAWzKmC1vF2EJP8aDeoC4AqNLFTfbRVyvEua4cgwGscDyALNs',
   'pubkey': '02778d1b2cbefa1e26eee707ebe5db36b4064e3c489f1915be1c7151df20b9bc5f'},
  {'path': "m/44'/0'/0'/0/1",
   'address': '1M895btaARGZWCgnemto7isDnwsJNbwzCH',
   'privkey': 'KxcFwimJNtq1hVQ5Zkjb881M3LFjEyVaZ65AvHmzvj4xK8BqLd5o',
   'pubkey': '024a866e2bc1cc08f43b721b3315a5edc02159efb4f004333f4756bee37179cdee'},
  {'path': "m/44'/0'/0'/0/2",
   'address': '19jeEK2togdsW1YKQRgD1bR6FNVbqVftJp',
   'privkey': 'L4Us4AqCevQALtv8UP38titF9eKBohj3fBXbmGyW4FNSAAF9srSj',
   'pubkey': '0231556833e8d7122ce9894221f5d6c69c86e808a72f68e96b4ea7c1c0d48b1df5'},
  {'path': "m/44'/0'/0'/0/3",
   'address': '18JKxbWV8D9zoiZgMzifCEuW65UeESvG6Y',
   'privkey': 'L1Mm2RNySPW93g7DQjMAWjP7W4BA5GnrBqmkq8zNdqmqv6vWPUJD',
   'pubkey': '036e5b7cb8aadc3cf5c1874f09e94d675549e8cb7c696a33333b1cb9272acaa8e6'},
  {'path': "m/44'/0'/0'/0/4",
   'address': '

In [11]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, private_key):
    if coin == ETH:
        return Account.privateKeyToAccount(private_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(private_key)

In [12]:
# Test function and get private key for ETH and BTCTEST
ETH_key = priv_key_to_account(ETH, eth_privatekey)

BTCTEST_key = priv_key_to_account(BTCTEST, btc_privatekey) 

print(ETH_key)
print(BTCTEST_key)

<PrivateKeyTestnet: mfys3aWwCHLUpEujadCpvzxRh2w1jr6v8k>


In [14]:
# Create a function called `create_raw_tx` that creates an unsigned transaction appropriate metadata.
def create_raw_tx(coin, account, recipient, amount):
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas({"from": account.address, "to": recipient, "value": amount})
        return {
        "from": account.address,
        "to": recipient,
        "value": amount,
        "gasPrice": w3.eth.gasPrice,
        "gas": gasEstimate,
        "nonce": w3.eth.getTransactionCount(account.address)
    }
    
    if coin == BTCTEST: 
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

In [15]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin, account, recipient, amount):
    tx = create_raw_tx(coin, account, recipient, amount)
    signed_tx = account.sign_transaction(tx)
    if coin == ETH:
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        return result.hex()
    elif coin == BTCTEST:
        return NetworkAPI.broadcast_tx_testnet(signed_tx)


In [21]:
# After funding balance with bitcoin (see screenshot), reconfirm current balance
btc_key = wif_to_key(btc_privatekey)
btc_key.get_balance('btc')

'0.0003'

In [22]:
# Send transaction to another account using send_tx function
# Transaction will be sent to a second account that has been created in Step 3 - see .txt file
send_tx(BTCTEST, BTCTEST_key, 'n1YhAstdvCLpE2fQPaBRjUXWsu1AjzJG3n', 0.0002)

In [23]:
# Using private key of the second account - confirm if the coins have arrived 
# Alternatively, refer to the attached screenshot No. 4
btc_key_two = wif_to_key('cTpbj5kmLAmrvZkM67EDRXcn1UiSJNjzaCutXDwX9jeZVVRQUUdu')
btc_key_two.get_balance('btc')

'0.0002'